<a href="https://colab.research.google.com/github/tktmyd/ipynbs/blob/main/fdm/FDM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 地震波数値シミュレーション入門

このノートブックでは，弾性体の運動方程式と線形弾性体の構成関係式を前提として，もっとも簡単な1次元の地震波伝播問題をスタガードグリッド差分法による数値シミュレーションを体験します．以下では座学と数値計算の実践や可視化が繰り返されますので，まず必要となるPythonライブラリのインポートをしておきましょう．その後，文章部分を学び，その下にあるコードを順に実行していくことで，地震波伝播の数値シミュレーションがどういうものかを体験してもらえるようになっています．

なお，この一連のノートブックでは，数値計算のためのライブラリとして [numpy](https://numpy.org) が，可視化ツールとして [PyGMT](https://www.pygmt.org/latest/) がそれぞれ使われています．NumPyについては多くの環境でインストール済みだと思います．PyGMTについては，たとえば[ここ](https://tktmyd.github.io/pygmt-howto-jp/install.html)にminiforgeを用いた簡易な環境構築の例があります．ライブラリの読み込みは，必要に応じてノートブックごとに，それぞれの先頭で行います．

## はじめに：弾性論と数値シミュレーション

地震波が震源から固体地球内部を広がっていく **地震波動伝播** の問題は，**弾性体の運動方程式**

$$
\rho \frac{\partial^2 u_i}{\partial t^2} = \sum_{j=1}^3 \frac{\partial \sigma_{ij}}{\partial x_j} + f_i
$$

と，応力テンソルとひずみテンソルとを関係づける **構成関係式** （ここでは**線形等方弾性体**を仮定する）

$$
\sigma_{ij} = \delta_{ij} \lambda \sum_{k=1}^3 \varepsilon_{kk} + 2 \mu \varepsilon_{ij}
$$

およびひずみテンソルの定義

$$
\varepsilon_{ij} = \frac{1}{2} \left( \frac{\partial u_i}{\partial x_j} + \frac{\partial u_j}{\partial x_i} \right)
$$

によって記述されます．ここで$\rho$は質量密度，$u_i$は弾性体の変位ベクトル，$f_i$は体積力のそれぞれ第$i$成分，$\sigma_{ij}$ と $\varepsilon_{ij}$は応力およびひずみテンソルの第$i,j$成分，$\lambda$と$\mu$はLaméの係数で，$P$波速度 $\alpha$, $S$波速度 $\beta$ と，

$$
\alpha = \sqrt{\frac{\lambda+2\mu}{\rho}}, \quad
\beta = \sqrt{\frac{\mu}{\rho}}
$$

という関係にあります．

地震（断層運動）は 体積力 $\boldsymbol{f}$ に相当する項で（ダブルカップル型の震源として）表され，地球内部の不均質構造は質量密度 $\rho$ と $\lambda$と$\mu$はLaméの係数 $\lambda$ と $\mu$（あるいは等価なことですが，地震波速度 $\alpha$ と $\beta$）が位置によって異なる値を取る，すなわち場所 $\boldsymbol{x}$ の関数であることによって表現されます．

このように，震源と地球内部構造とが与えられたとき，運動方程式と構成関係式を連立偏微分方程式としてもし**解く**ことができれば，すなわち$\boldsymbol{u}(\boldsymbol{x},t)$ のように場所 $\boldsymbol{x}$ と時間 $t$ の関数として弾性体の変位ベクトルを求めることができれば，与えられた震源によって既知の地球内部構造を伝わる地震波が完全に再現できた，ということになります．

しかし，弾性体の運動方程式はかなり複雑な方程式で，たとえば地球内部構造が完全に均質（密度やLaméの係数が一定）のようなきわめて限られた条件でしか，この方程式を解析的に解くことはできません．そこで，ここでは微分方程式に対してある **近似** を施すことで，この問題をコンピュータで直接答えを求められるような代数的な（四則演算による）計算のみによって解く，ということを考えます．このようなアプローチを一般に **数値シミュレーション** とか **数値計算** と呼びます．

## 問題設定と基礎方程式

いきなり現実世界の3次元空間における地震波のシミュレーションは，アルゴリズムも煩雑になりますし，大規模な計算機も必要になってしまいます．
そこでここでは，まず運動方程式と構成関係式を数値計算に都合のよい形に変形し，その次に，特別な場合として「1次元の方程式」を作ります．それに対して数値シミュレーションの技法を当てはめていくことにしましょう．

まずは，運動方程式について，変位のかわりに速度 $v_i = \partial u_i / \partial t$ を独立変数とします．もう一つ，構成関係式にはひずみの定義を代入してしまい，そのうえで両辺を1階時間で微分します．

運動方程式については，加速度ベクトル成分を変位速度で表すのですから，変位の2階微分のかわりに速度の1階微分を用いて

$$
\rho \frac{\partial v_i}{\partial t} = \sum_{j=1}^3 \frac{\partial \sigma_{ij}}{\partial x_j} + f_i
$$

となります．一方，構成関係式にひずみテンソルの定義を代入し，さらに時間微分すると，

$$
\frac{\partial \sigma_{ij}}{\partial t} = \delta_{ij} \lambda \sum_{k=1}^3 \frac{\partial v_k}{\partial x_k} + \mu \left( \frac{\partial v_i}{\partial x_j} + \frac{\partial v_j}{\partial x_i}  \right)
$$

が得られました．このように，変位速度と応力の時間微分に関する連立方程式を用いた表現のことを， **速度・応力型**の方程式と呼びます．運動方程式は左辺が変位速度の時間微分で右辺が応力の空間微分，構成関係式は左辺が応力の時間微分で右辺が変位速度の空間微分，と，きわめて対称性の高い形になっており，数値シミュレーションをするうえで解きやすいのです．

さらにここでは，1次元近似をします．空間微分のうち$x_1$微分以外はすべて$0$としてしまうと，以下の3つの組の方程式が得られます．

$$
\begin{align}
    &\rho \frac{\partial v_1}{\partial t} = \frac{\partial \sigma_{11}}{\partial x_1} , \quad \frac{\partial \sigma_{11}}{\partial t} = \left(\lambda+ 2\mu \right)\frac{\partial v_1}{\partial x_1}
    \\
    &\rho \frac{\partial v_2}{\partial t} = \frac{\partial \sigma_{12}}{\partial x_1} , \quad \frac{\partial \sigma_{12}}{\partial t} = \mu \frac{\partial v_2}{\partial x_1}    
    \\
    &\rho \frac{\partial v_3}{\partial t} = \frac{\partial \sigma_{13}}{\partial x_1} , \quad \frac{\partial \sigma_{13}}{\partial t} = \mu \frac{\partial v_3}{\partial x_1}    
\end{align}
$$

それぞれの式は，運動方程式1本と構成関係式1本の組で， **閉じた**（すなわち原理的には他の成分の式を用いずに解くことができる）形になっています．しかも，3本とも形式的には同じで，


$$
\rho \frac{\partial v}{\partial t} = \frac{\partial \sigma}{\partial x}, \quad
\frac{\partial \sigma}{\partial t} = G \frac{\partial v}{\partial x}
$$

という形をしています．$G$は**剛性率**に相当するもので，$\lambda+2\mu$ もしくは $\mu$ の値を取ります．また，空間方向は1方向しか現れないため，$x_1$のかわりに単に$x$と表記しました．ここではこれを1次元の方程式系，として受け入れましょう．